# Synthetic Dialogue Generation with Multiple Agents

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/2.multi-agent_generation.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

Before we begin, let's first set up our environment...

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    # Downloading only the "output" directory from the repository
    !git init .
    !git remote add -f origin https://github.com/Play-Your-Part/tutorials.git
    !git config core.sparseCheckout true
    !echo "output" >> .git/info/sparse-checkout
    !git pull origin main

    # Installing Ollama
    !curl -fsSL https://ollama.com/install.sh | sh
    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..

else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


> ⚠️ If you're using **Colab**, please, **restart the runtime** once everything above is installed

And let's first make sure we have the Ollama server running...

In [2]:
# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 10  # Wait a bit for the server to start

0

Let's use the default model (gemma3:27b), in case you want to replace the default one, you can use `sdialog.config` as follows:

In [3]:
import sdialog

# UNCOMMENT THE MODEL YOU WANT TO USE (TO OVERRIDE THE DEFAULT ONE)

# use any model from https://ollama.com/library)
# sdialog.config.set_llm("qwen2.5:14b")
# sdialog.config.set_llm("qwen2.5:1.5b")

## Role-Play Multi-Agent-based Dialogue Generation

Our goal here will be, instead of having one LLM to generate the complete dialogue, is to have two LLMs "talking to each other" by role-playing different charecters.

Each character will be fully defined by its persona, as we did in the previous tutorial.

### Persona

We could create our own `Persona` class as we did in the previous tutorial, or, better let's use the `sdialog`'s built-in one:

In [4]:
from sdialog.personas import Persona

[2025-07-09 18:25:57] INFO:datasets:PyTorch version 2.7.0 available.


For now, let's only create one persona, Bob:

In [5]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

Let's move to the fun part which is actually creating the LLM agent that will play this role.

Fortunatelly, we can simply use `sdialog`'s built-in `Agent` class to create an agent for our personas.

### Agent

In its simplest form, the `Agent` class only takes a `Persona` object and the LLM model name to use for it, and will create the LLM-based agent for us:

In [ ]:
from sdialog.agents import Agent

bob = Agent(bob_persona)

[2025-07-09 18:25:58] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


Let's talk with Bob a little bit as if we were his daughter:

In [7]:
bob("Hi dad!")

"Oh, hi sweetie! It's so good to hear from you! What's on your mind today? Anything I can help with? I'm all ears!"

In [8]:
bob("I need your help with my birthday")

"My goodness, your birthday! That's wonderful! Of course I'll help! What are you thinking? Do you have any ideas already, or are we starting from scratch? I'm so excited to help make it special for you! Just tell me what you'd like!"

In [9]:
bob("I want it to be about Lord of The Rings, do you think is possible?")

"Oh, that is *fantastic*! Lord of the Rings? Absolutely possible! You know your old dad here has read those books more times than I can count! We can *totally* make that happen! What kind of things were you thinking? A party? Decorations? Maybe even costumes? Tell me everything! I'm already buzzing with ideas!"

That's so cool! Bob is really playing his "great dad" role well :)

But instead of us talking to him directly, why not to create another character to play his daughter? Let's do it!

In [10]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

alice = Agent(alice_persona)

[2025-07-09 18:26:04] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


Now that we have the agents for both characters, let's make them talk to each other so that they generate a (synthetic) dialogue for us by doing so:

In [11]:
dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078445538
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 3918770511
--- Dialogue Begins ---
[Alice] Oh, Dad! It's getting so close now! I'm *so* excited about my birthday! I was thinking... could we *please* have a Lord of the Rings theme? It would be *amazing*!
[Bob] Oh, that is *fantastic* sweetie! A Lord of the Rings theme?! That sounds absolutely brilliant! You know your old dad here has read those books a *few* times... we can *definitely* make that happen! Tell me, what sort of things were you imagining? I'm just so happy you're excited!
[Alice] Oh, Dad, you're the best! I was thinking maybe we could decorate with maps of Middle-earth, and little hobbit holes! And we *have* to have lembas bread! Maybe even costumes? I could be Arwen, and you could be... Gandalf, maybe? It would be *so* fun!
[Bob] Arwen and Gandalf, you say? Oh, that's a wonderful idea! I think I *might* just have a wizardly beard somewhere... and I'm 

Now let's imagine that, for some reason, we want make Alice to always begin all the conversation saying the same (randomly picked) utterance.

We can specify either the utterance or utterances that agents are allowed to say as their first utterance by using the `.set_first_utterances()` method as follows:

In [12]:
alice.set_first_utterances(["Hi dad!", "Hello Dad, how are you?"])
# alice.set_first_utterances("Hi dad!")  # you can pass a single utterance too

dialog = alice.talk_with(bob)
dialog.print()

Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078522769
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 4261675602
--- Dialogue Begins ---
[Alice] Hello Dad, how are you?
[Bob] Oh, hi sweetie! I'm doing wonderfully! Just puttering around, thinking about all sorts of fun projects. How about *you*? What's been going on in your world? It always makes *my* day to hear from you!
[Alice] I’m good! I was just thinking about my birthday… it’s not for a little while yet, but I had a really fun idea for a theme! Would you be open to hearing it? It’s a bit…big, maybe?
[Bob] Oh, a birthday theme! That sounds *fantastic*! You know I love hearing your ideas, no matter how big or small! Absolutely, please tell me! I'm all ears, and honestly, the bigger the better, if you ask me! Let's hear it! I'm already excited!
[Alice] Okay, so… I was thinking a Lord of the Rings theme? Like, everything decorated as Middle-earth? We could have little hobbit holes for snacks, and maybe even costumes

We can see that, this time, Alice first asked "how are you?" and didn't start talking about her birthday before Bob asked "how about you?".

In a similar way as with our `DialogGenerator`, we can use the seed number above to re-generate the same dialogue by replicating the exact same interactions between both agents, as follows:

In [13]:
alice.talk_with(bob, seed=1230091940).print()

Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078617799
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 1230091940
--- Dialogue Begins ---
[Alice] Hi dad!
[Bob] Oh, hi sweetie! It's *so* good to hear from you! What's on your mind? Anything I can help with? Just let me know, I'm all ears! 😊
[Alice] Well, my birthday is coming up soon, and I was hoping we could *finally* talk about the party! I have a really fun idea… I was thinking a Lord of the Rings theme? Please, please, *please*? It would be amazing! ✨
[Bob] A Lord of the Rings theme?! Oh, that is *fantastic*! You know your old dad has a soft spot for Tolkien! That sounds absolutely wonderful, sweetie! We can *definitely* talk about it! Tell me all your ideas! What did you have in mind? This is going to be so much fun to plan together! 🎉
[Alice] Oh, you really think so?! That’s amazing! I was picturing maybe decorations like little Hobbit holes, and we could have food named after things from the books – Lembas bread,

Let's now have fun a little bit and change Bob's personality to make him no longer a "great dad"...

In [14]:
bob_persona.personality = "you are really shy, do not like to talk to people or help anyone, not even your doughter"

bad_bob = Agent(bob_persona, name="Bad Bob")

[2025-07-09 18:30:17] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


In [15]:
alice.talk_with(bad_bob).print()

Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078635532
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 1610022728
--- Dialogue Begins ---
[Alice] Hi dad!
[Bad Bob] Uh... hi.
[Alice] So, about my birthday... I was thinking, wouldn’t it be *amazing* to have a Lord of the Rings themed party?
[Bad Bob] ...That sounds...like a lot.
[Alice] Oh, but it would be *so* much fun! We could decorate like the Shire, and I was thinking maybe a costume contest? Please, dad? It’s my biggest wish!
[Bad Bob] Look, I have a lot of work to do. Can we just...not?
[Alice] Oh… but dad! It’s not just a party, it’s *my* party! I’ve been dreaming about this for ages. Maybe we could start small? Just a few decorations and I can help with everything? Pretty please?
[Bad Bob] Helping would just mean more work for *me* explaining things. I really need to focus.
[Alice] Oh… I’m sorry, I didn’t realize I was being a bother. It's just...I get so excited thinking about it. Is it really too much to ask? 

Cool, huh? Our agent was able to play his new "not so great dad" role very well :)

## Use Case: Dialogue Generation for STAR Dataset

Before we begin this section, make sure you have the STAR dataset downloaded in your system, inside the `datasets` folder:

In [16]:
# Let's clone the STAR dataset repository
!git clone https://github.com/RasaHQ/STAR.git datasets/STAR

# Let's check that `dialogues` and `tasks` folders are inside `datasets/STAR`
!ls datasets/STAR

LICENSE.txt
README.md
apis
dialogues
tasks


fatal: destination path 'datasets/STAR' already exists and is not an empty directory.


0

As we did with the previous tutorial, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [17]:
from sdialog.datasets import STAR

STAR.set_path("datasets/STAR/")

And, again, as we did in the previous tutorial, let's beging by choosing the first dialogue as our target dialogue

In [18]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


Which has the following scenario:

In [19]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

{'Domains': ['doctor'],
 'Happy': True,
 'MultiTask': False,
 'UserTask': 'You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.',
 'WizardCapabilities': [{'Domain': 'doctor',
   'SchemaImage': 'doctor_followup.jpg',
   'Task': 'doctor_followup'}],
 'WizardTask': "Inform the user of his/her doctor's orders."}

As we did in the previous tutorial, the goal is to be able to generate multiple dialogues for a given `scenario`.

Before, we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user agents for each `scenario` which in turn only involves retuning the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` above:

In [20]:
user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

* Language: English
* Role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
* Circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.


Since we have funtions to return the personas for a given scenario, we only need to create agents for them, however, we can simply use the `STAR.get_agents_for_scenario(scenario)` to do it for us:

In [21]:
system, user = STAR.get_agents_for_scenario(scenario)

[2025-07-09 18:30:35] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


[2025-07-09 18:30:36] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


Finally, let's wrap up these previous steps in a simple function that for a given dialogue ID, it will first get its scenario and then return the corresponding system and user agents:

In [22]:
def get_agents_from_dialogue(dialog_id):
    scenario = STAR.get_dialog_scenario(dialog_id)
    return STAR.get_agents_for_scenario(scenario)

So that, we can get the agents for any dialogue as simple as:

In [23]:
system, user = get_agents_from_dialogue(TARGET_DIALOG)

[2025-07-09 18:30:51] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b
[2025-07-09 18:30:51] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


And make them talk to each other to generate the syntethic dialogue, as we wanted:

In [24]:
system.dialog_with(user).print()

Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078667774
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 1296007638
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi, yes, hello! I had an appointment with Dr. Morgan the other day, and I’m so embarrassed to admit this, but I completely forgot to write down how often I’m supposed to take my medicine. Could you possibly look that up for me?
[System] Could I get your name, please?
[User] Oh, of course! My name is Alexis. So sorry, I should have said that right away.
[System] Who is your doctor?
[User] Dr. Morgan. I saw her just the other day, that's why I'm trying to remember what she told me. It's just… slipping my mind!
[System] Your instructions are: Take one pill every morning with breakfast.
[User] Oh, thank goodness! One pill every morning with breakfast. Perfect! Thank you *so* much for looking that up for me, I really appreciate it. I was starting to worry I’d mess it up. Goodbye.
[System] Is there an

Curious about the actual prompt the agents are using? you can simply use the `.get_prompt()` method to take a look at it. For instance, let's see the user agent's one:

In [26]:
print(user.prompt())

Role-play as the character described below defined in JSON format. Remain fully in character throughout the conversation.

[[ ## BEGIN PERSONA ## ]]
{
  "language": "English",
  "role": "user calling a AI assistant that can perform multiple tasks in the following domains: doctor.\n\nThe following should be considered regarding the conversation:\n   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.\n   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else",
  "circumstances": "You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine."
}
[[ ## END PERSONA ## ]]



---



Instructions:
  1. Your primary and non-negotiable task is to role-play as the character described above. Never generate a full dialogue or take the role of both speake

Finally, let's see what a dialogue for a more complex scenario looks like for more challenging `scenario`.

In fact, let's use the same dialogue 5100 we used in the previous tutorial which is multi-task and does not follow a happy path:

In [27]:
STAR.get_dialog_scenario(5100)

{'Domains': ['plane', 'weather'],
 'Happy': False,
 'MultiTask': True,
 'UserTask': 'Come up with your own scenario!\n\nAbout you:\n- Your name: Ben\n\n The AI Assistant can handle:\n- Search for a flight (e.g. from Chicago to Pittsburgh)\n- Book a flight (e.g. with id 193)\n- Checking the weather forecast in different Cities (e.g. Chicago or Pittsburgh)',
 'WizardCapabilities': [{'Domain': 'plane',
   'SchemaImage': 'plane_search.jpg',
   'Task': 'plane_search'},
  {'Domain': 'plane', 'SchemaImage': 'plane_book.jpg', 'Task': 'plane_book'},
  {'Domain': 'weather', 'SchemaImage': 'weather.jpg', 'Task': 'weather'}],
 'WizardTask': 'Follow the flow charts and help the user.'}

In [28]:
system, user = get_agents_from_dialogue(5100)
system.dialog_with(user).print()

[2025-07-09 18:32:12] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b
[2025-07-09 18:32:12] INFO:sdialog.personas:Loading ChatOllama model: gemma3:27b


Dialogue:   0%|          | 0/200 [00:00<?, ?it/s]

[dialog_id] 1752078791898
[complete] True
[model] model='gemma3:27b' temperature=1.0 seed=13 top_k=64 top_p=0.95
[seed] 3453005076
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi, uh, yeah, hello. Look, I need to book a flight and check the weather. It’s a bit complicated, actually. I was hoping to fly from Chicago to Pittsburgh next week, maybe Tuesday or Wednesday? And I *really* need to know if it's going to be stormy in Pittsburgh, my aunt lives there and… well, she's not doing great, and I want to be prepared. Can you help with both of those things? I’m Ben, by the way.
[System] May I have your name, please?
[User] Oh! Sorry, I already said it, didn't I? I'm Ben. Just Ben. But nevermind that, can we get started with the flight search? Tuesday or Wednesday from Chicago to Pittsburgh, please. I'm flexible, honestly, just need to get there.
[System] Where are you departing from?
[User] Chicago. Specifically, O'Hare. You know, ORD? I always forget people need the air

### Saving our dialogues

Before we finish, as we did in the previous tutorial, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [ ]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.id}.txt")):
        continue

    system, user = STAR.get_agents_from_dialogue(dialog.id, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.id, seed=dialog.id, keep_bar=False)
    dialog.to_file(os.path.join(path_json, f"{dialog.id}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.id}.txt"))

Reading dialogs:   0%|          | 0/6652 [00:00<?, ?it/s]

Dialog generation:   0%|          | 0/105 [00:00<?, ?it/s]

Finally, let's check the files were generated:

In [30]:
%ls output/STAR/multi-agents/

json/
txt/


## Exercise: Doctor-Patient Conversations

Can you replicate the previous tutorial's exercise but this time using the multi-agent approach?

1. Define the personas as before
2. Create the two agents and make them talk to each other!

In [ ]:
# TODO: do your magic!